## Scikit-learn / ML attempts

Specifically:
* **Classification** of **genre** based on numerical data, gender, ...
* **Classification** of **gender** based on numerical data, genre, ... - watch out, very unbalanced
* **Classification** of **artist** based on numerical data, gendre, gender?, ...
* **Classification** of **genre** for groupby mean data
* **Regression** of **unique_words, total_words, topic count** etc
* Same things for decade data except no gender and decade classification instead of genre 

In [ ]:
import pandas as pd
import numpy as np
# Metrics for Evaluation of model Accuracy and F1-score
from sklearn.metrics  import f1_score,accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
# regression metrics

# For splitting of data into train and test set
from sklearn.model_selection import train_test_split
# for categorical columns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# classification
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# regression
from sklearn.linear_model import LinearRegression
from xgboost.sklearn import XGBRegressor
from sklearn.svm import SVR

import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
counts_df = pd.read_csv('./dataframes/with_counts/combined_count.csv', index_col = 0)
genre_mean_df = pd.read_csv('./dataframes/group_stats/genre_mean_df.csv', index_col = 0)
artist_genre_mean_df = pd.read_csv('./dataframes/group_stats/mean/combined_artists.csv', index_col = 0)
decades_counts_df = pd.read_csv('./decade_dataframes/decades_counts.csv', index_col = 0)
decades_mean_df = pd.read_csv('./decade_dataframes/decade_groupby_mean.csv', index_col = 0)

In [ ]:
counts_df.head()

In [ ]:
counts_df.info()

In [ ]:
counts_df.columns

### Select columns suitable for our ML tasks

#### check for NaNs

In [ ]:
counts_df.isna().sum().sort_values(ascending=False)

#### should we drop the rows with nulls or not?
* month and day most likely provide little value
* year could be usedful though, lets try dropping rows with null year
* a better option could be to impute the year based on the artist or even manually fix these by checking the release dates

In [ ]:
counts_classif_cols = ['Year',
                      'Pageviews', 'featured_count', 'producer_count', 'writer_count',
                      'gender', 'genre', 'unique_words', 'total_words', 
                       'manual_love_count',
                       'manual_money_count', 'manual_violence_count', 'manual_drugs_count',
                       'manual_gendered_count', 'manual_sadness_count', 'manual_joy_count',
                       'manual_yes_count', 'manual_no_count',
                       'sentiment', 'emotion'
                      ]
counts_classif_cols_no_gender = [col for col in counts_classif_cols if col != 'gender']
counts_classif_cols_no_genre = [col for col in counts_classif_cols if col != 'genre']
counts_classif_cols_no_sentiment = [col for col in counts_classif_cols if col != 'sentiment']
counts_classif_cols_no_emotion = [col for col in counts_classif_cols if col != 'emotion']

In [ ]:
classif_counts_df = counts_df[[*counts_classif_cols]]

#### dropna

In [ ]:
classif_counts_df.dropna(inplace=True)

In [ ]:
classif_counts_df.shape

In [ ]:
def train_classify_target(df, target_col, model, train_size=0.8, task='class'):
    """
    Takes a dataframe, target column and model,
    plus optionally train_size and tak
    
    if task set to 'class' will calculate classification metrics
    if task set to 'regr' will calculate regression metrics!
    
    creates a train-test split,
    preprocesses the (standard scaling and one-hot-encoding) pipeline,
    calculates accuracy, f1, confusion matrix metrics,
    returns the trained model and metrics

    Caveats:
        * Expects a df without NaNs, you have to choose columns / rows to drop / keep first
        * The pipeline could be expanded

    To be expanded:
        * right now takes model, could possibly take model_type and hyperparams
    
    """
    y = df[target_col]
    X = df.drop([target_col], axis=1)

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=train_size, test_size=(1-train_size))

    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

    # Preprocessing transformers
    numerical_transformer = StandardScaler()
    
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

    # combined preprocessor
    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )

    # Bundle preprocessing and modeling code in a pipeline
    train_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Preprocessing of training data, fit model 
    train_pipeline.fit(X_train, y_train)

    # Preprocessing of validation data, get predictions
    preds = train_pipeline.predict(X_valid)

    # metrics
    if task == 'class':
        acc = accuracy_score(y_valid, preds)
        f1 = f1_score(y_valid, preds, average=None, labels=model.classes_)
        cm = confusion_matrix(y_valid, preds, labels=model.classes_)
        labels = model.classes_

        return acc, f1, cm, labels, model

    if task == 'regr':
        mae = mean_absolute_error(y_valid, preds)
        r2 = r2_score(y_valid, preds)
        return mae, r2, model

In [ ]:
def display_confusion_matrix(cm, labels):
    disp = ConfusionMatrixDisplay(confusion_matrix = cm,
                             display_labels=labels
                             )
    disp_plot = disp.plot()
    plt.show()

## Classify for genre

### by counts_df
classif_counts_df, SVC, target='genre'

In [ ]:
model = SVC()

In [ ]:
acc, f1, cm, labels, trained_model = train_classify_target(
    df = classif_counts_df,
    target_col = 'genre',
    model = model
)

In [ ]:
display_confusion_matrix(cm, labels)

In [ ]:
print(acc)
print(f1)

### Classify for gender

In [ ]:
model = SVC()
acc, f1, cm, labels, trained_model = train_classify_target(
    df = classif_counts_df,
    target_col = 'gender',
    model = model
)
display_confusion_matrix(cm, labels)
print(acc)
print(f1)

### Classify for sentiment

In [ ]:
model = SVC()
acc, f1, cm, labels, trained_model = train_classify_target(
    df = classif_counts_df,
    target_col = 'sentiment',
    model = model
)
display_confusion_matrix(cm, labels)
print(acc)
print(f1)

### Classify for emotion

In [ ]:
model = SVC()
acc, f1, cm, labels, trained_model = train_classify_target(
    df = classif_counts_df,
    target_col = 'emotion',
    model = model
)
display_confusion_matrix(cm, labels)
print(acc)
print(f1)

### Classify for artist

In [ ]:
counts_classif_cols_artist  = [*counts_classif_cols, 'Artist']
counts_classif_cols_artist

In [ ]:
classif_counts_artist_df = counts_df[[*counts_classif_cols_artist]]
classif_counts_artist_df.dropna(inplace=True)

In [ ]:
model = SVC()
acc, f1, cm, labels, trained_model = train_classify_target(
    df = classif_counts_artist_df,
    target_col = 'Artist',
    model = model
)
display_confusion_matrix(cm, labels)
print(acc)
print(f1)

### classify genre from groupby mean data
hold on though - it has to be artist means, if you just get genre means, thats 4 rows of data

In [ ]:
# genre_mean_df.head().columns

In [ ]:
# genre_mean_df['genre'] = genre_mean_df.index

In [ ]:
artist_genre_mean_df.columns

In [ ]:
no_artist_genre_mean_df = artist_genre_mean_df.drop(['Artist'], axis=1)

In [ ]:
model = SVC()
acc, f1, cm, labels, trained_model = train_classify_target(
    df = no_artist_genre_mean_df,
    target_col = 'genre',
    model = model
)
display_confusion_matrix(cm, labels)
print(acc)
print(f1)

### classify for decade from decades counts df

In [ ]:
decades_counts_df.columns

In [ ]:
decades_classif_cols = [
    'Year',
    'Pageviews',
    'featured_count', 'producer_count', 'writer_count',
    'unique_words', 'total_words',
    'manual_love_count', 'manual_swears_count',
   'manual_money_count', 'manual_violence_count', 'manual_drugs_count',
   'manual_gendered_count', 'manual_sadness_count', 'manual_joy_count',
   'manual_yes_count', 'manual_no_count', 'decade', 'sentiment'
]

In [ ]:
decades_classif_df = decades_counts_df[[*decades_classif_cols]]

In [ ]:
decades_classif_df.isna().sum().sort_values(ascending=False)

In [ ]:
decades_classif_df.dropna(inplace=True)

In [ ]:
decades_classif_df.shape

In [ ]:
model = SVC()
acc, f1, cm, labels, trained_model = train_classify_target(
    df = decades_classif_df,
    target_col = 'decade',
    model = model
)
display_confusion_matrix(cm, labels)
print(acc)
print(f1)

## Regression

### regress for unique_words on counts

In [ ]:
model = SVR()
mae, r2, trained_model = train_classify_target(
    df = classif_counts_df,
    target_col = 'unique_words',
    model = model,
    task = 'regr'
)
# display_confusion_matrix(cm, labels)
print(mae)
print(r2)

### regress for manual_love_count on counts

In [ ]:
model = SVR()
mae, r2, trained_model = train_classify_target(
    df = classif_counts_df,
    target_col = 'manual_love_count',
    model = model,
    task = 'regr'
)
# display_confusion_matrix(cm, labels)
print(mae)
print(r2)

#### TODO: testing some predictions
would need to pass the sample to test on on the same preprocessing steps taken during trainign
probably not that hard with a pipeline
and could be written as a pretty universal function


## Manual Tests

### Get columns of type object and one-hot encode

#### object_cols

In [ ]:
# Get list of categorical variables
s = (classif_counts_df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

### Example for genre classification

##### select columns

In [ ]:
genre_classif_X = classif_counts_df[[*counts_classif_cols_no_genre]]
genre_classif_y = classif_counts_df[['genre']]

In [ ]:
genre_classif_X

In [ ]:
genre_classif_y

#### train-test split

In [ ]:
X_train_genre, X_test_genre, y_train_genre, y_test_genre = train_test_split(
    genre_classif_X,
    genre_classif_y,
    train_size=0.8,
    test_size=0.2,
    shuffle=True
)

In [ ]:
object_cols_no_genre = ['gender', 'sentiment', 'emotion']

In [ ]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train_genre = pd.DataFrame(OH_encoder.fit_transform(X_train_genre[[*object_cols_no_genre]]))
OH_cols_test_genre = pd.DataFrame(OH_encoder.transform(X_test_genre[[*object_cols_no_genre]]))

# One-hot encoding removed index; put it back
OH_cols_train_genre.index = X_train_genre.index
OH_cols_test_genre.index = X_test_genre.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train_genre = X_train_genre.drop(object_cols_no_genre, axis=1)
num_X_test_genre = X_test_genre.drop(object_cols_no_genre, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train_genre = pd.concat([num_X_train_genre, OH_cols_train_genre], axis=1)
OH_X_test_genre = pd.concat([num_X_test_genre, OH_cols_test_genre], axis=1)

# !!
# Ensure all columns have string type
OH_X_train_genre.columns = OH_X_train_genre.columns.astype(str)
OH_X_test_genre.columns = OH_X_test_genre.columns.astype(str)


In [ ]:
model_genre_1_preds = model_genre_1.predict(OH_X_test_genre)

In [ ]:
f1_score(y_test_genre, model_genre_1_preds, average=None)

In [ ]:
accuracy_score(y_test_genre, model_genre_1_preds)

In [ ]:
y_test_genre

In [ ]:
genres = list(y_test_genre.genre.unique())

In [ ]:
cm = confusion_matrix(y_test_genre, model_genre_1_preds, labels=model_genre_1.classes_)

In [ ]:
cr = classification_report(y_test_genre, model_genre_1_preds, labels=model_genre_1.classes_)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix = cm,
                             display_labels=model_genre_1.classes_
                             )

In [ ]:
disp.plot()

#### numerical cols scaling test

In [ ]:
numerical_cols = [cname for cname in X_train_genre.columns if X_train_genre[cname].dtype in ['int64', 'float64']]

# Remove categorical columns (will replace with one-hot encoding)
num_X_train_genre = X_train_genre.drop(object_cols_no_genre, axis=1)
num_X_test_genre = X_test_genre.drop(object_cols_no_genre, axis=1)

# scaling
scaler = StandardScaler().fit(num_X_train_genre)
num_X_train_genre = pd.DataFrame(scaler.transform(num_X_train_genre))
num_X_test_genre = pd.DataFrame(scaler.transform(num_X_test_genre))

# Add one-hot encoded columns to numerical features
OH_X_train_genre = pd.concat([num_X_train_genre, OH_cols_train_genre], axis=1)
OH_X_test_genre = pd.concat([num_X_test_genre, OH_cols_test_genre], axis=1)

# !!
# Ensure all columns have string type
OH_X_train_genre.columns = OH_X_train_genre.columns.astype(str)
OH_X_test_genre.columns = OH_X_test_genre.columns.astype(str)

In [ ]:
model_genre_1 = SVC()
model_genre_1.fit(OH_X_train_genre, y_train_genre)

model_genre_1_preds = model_genre_1.predict(OH_X_test_genre)

acc = accuracy_score(y_test_genre, model_genre_1_preds)
cm = confusion_matrix(y_test_genre, model_genre_1_preds, labels=model_genre_1.classes_)

print(acc)

disp = ConfusionMatrixDisplay(confusion_matrix = cm,
                             display_labels=model_genre_1.classes_
                             )

### Pipelines test, cause this is bullshit

like we're just starting

In [ ]:
counts_df_pre_pipe = counts_df[counts_classif_cols]
counts_df_pre_pipe.dropna(inplace=True)

y = counts_df_pre_pipe.genre
X = counts_df_pre_pipe.drop(['genre'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

#### Important: get the categorical columns only after removing the target feature
- makes your life so much easier to just do it in that order for every feature

In [ ]:
categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# training pipeline
# Bundle preprocessing and modeling code in a pipeline
model = SVC()

train_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

train_pipeline.fit(X_train, y_train)

preds = train_pipeline.predict(X_valid)

acc = accuracy_score(y_valid, preds)
cm = confusion_matrix(y_valid, preds, labels=model.classes_)

print(acc)

disp = ConfusionMatrixDisplay(confusion_matrix = cm,
                             display_labels=model.classes_
                             )

disp.plot()